In [1]:
#!pip install lightgbm
#!pip install optuna
#!pip memoized_property

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
from income.ml_trainer import LigthTrainer
from s3_utils import read_pd_from_parquet, write_pickle, read_pickle

In [4]:
import pandas as pd
import gc
from sklearn.model_selection import train_test_split
import optuna
from sklearn.metrics import mean_absolute_percentage_error as MAPE
from sklearn.metrics import mean_squared_error 

import lightgbm as lgbm
import joblib
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import optuna 
from optuna.visualization import plot_optimization_history, plot_param_importances
from optuna import Trial, visualization
from optuna.samplers import TPESampler

from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from statistics import mean
from s3_utils.s3_utils import write_parquet_from_pd, read_pd_from_parquet, start_logger,write_pickle,read_pickle

from s3_utils import read_pd_from_csv
from income.data_validation import quality_check,missing_threshold,df_optimized
pd.set_option('display.max_rows', 500)


import sys

    path = "glue_scripts/output"

Running script locally


In [7]:
df = pd.concat([aprobados, prospectos], ignore_index=True, sort=False)
df.shape

(27090, 50)

In [8]:
#del aprobados
del prospectos

df = df.drop_duplicates(subset="researchable_id", keep="last")
df = df.query("ingreso_neto_comprobado > 8000 & ingreso_neto_comprobado < 300000")
df = df.query("net_income_verified > 8000 & net_income_verified < 600000")
df = df[df["net_income_verified"]>=df["ingreso_neto_comprobado"]]

In [9]:
df = df_optimized(df)

optimized size by 51.0 % | 0.003732771 GB


In [10]:
#quality_check(df)

In [11]:
test_df = aprobados.query("ingreso_neto_comprobado > 8000 & ingreso_neto_comprobado < 200000").sample(3714)
train = df.loc[~df.researchable_id.isin(test_df.researchable_id),]
train.to_csv("data/train.csv")
test_df.to_csv("data/test.csv")

In [12]:
del df

for df in [test_df,train,aprobados]:
    df  = df_optimized(df)


In [13]:
target = "ingreso_neto_comprobado"
to_drop = ['researchable_id', 'estimate', 'declarativa',"ingreso_neto_comprobado"]
to_drop = to_drop + ["ingreso_neto_declarado"]
columnas = [col for col in test_df.columns if col not in to_drop]

## Ligthgbm

In [14]:
lgbm = LigthTrainer()
lgbm.X_train = train[columnas]
lgbm.y_train = train[target].clip(0,200000)
lgbm.X_test = test_df[columnas]
lgbm.y_test = test_df[target]

lgbm.optimize()
lgbm.evaluate()
lgbm.save_model("preds_declarado")

Training until validation scores don't improve for 20 rounds
Did not meet early stopping. Best iteration is:
[500]	valid_0's l2: 1.73711e+08
Training until validation scores don't improve for 20 rounds
Early stopping, best iteration is:
[334]	valid_0's l2: 1.7549e+08
Training until validation scores don't improve for 20 rounds
Early stopping, best iteration is:
[316]	valid_0's l2: 1.7242e+08
Training until validation scores don't improve for 20 rounds
Did not meet early stopping. Best iteration is:
[500]	valid_0's l2: 1.78111e+08
Training until validation scores don't improve for 20 rounds
Early stopping, best iteration is:
[287]	valid_0's l2: 1.75595e+08
Training until validation scores don't improve for 20 rounds
Did not meet early stopping. Best iteration is:
[500]	valid_0's l2: 1.7535e+08
Training until validation scores don't improve for 20 rounds
Did not meet early stopping. Best iteration is:
[498]	valid_0's l2: 1.72452e+08
Training until validation scores don't improve for 20 r

## Modelo % validado

In [15]:
validated_train = train.copy()
validated_train["target"] = (validated_train["ingreso_neto_comprobado"]/validated_train["net_income_verified"])
del validated_train["ingreso_neto_comprobado"]

validated_test = test_df.copy()
validated_test["target"] = (validated_test["ingreso_neto_comprobado"]/validated_test["net_income_verified"])
del validated_test["ingreso_neto_comprobado"]


lgbm = LigthTrainer()
lgbm.fit(validated_train,model_name="validado", to_drop=['researchable_id', 'estimate', 'declarativa'], target="target")
lgbm.X_train = validated_train[lgbm.columns]
lgbm.y_train = validated_train[lgbm.target]
lgbm.X_test = validated_test[lgbm.columns]
lgbm.y_test = validated_test[lgbm.target]

lgbm.optimize()
lgbm.evaluate()
lgbm.save_model("%_validado")

Training until validation scores don't improve for 20 rounds
Early stopping, best iteration is:
[30]	valid_0's l2: 426.675
Training until validation scores don't improve for 20 rounds
Early stopping, best iteration is:
[35]	valid_0's l2: 426.683
Training until validation scores don't improve for 20 rounds
Early stopping, best iteration is:
[154]	valid_0's l2: 426.678
Training until validation scores don't improve for 20 rounds
Early stopping, best iteration is:
[48]	valid_0's l2: 426.689
Training until validation scores don't improve for 20 rounds
Early stopping, best iteration is:
[37]	valid_0's l2: 426.681
Training until validation scores don't improve for 20 rounds
Early stopping, best iteration is:
[202]	valid_0's l2: 426.679
Training until validation scores don't improve for 20 rounds
Early stopping, best iteration is:
[28]	valid_0's l2: 426.669
Training until validation scores don't improve for 20 rounds
Early stopping, best iteration is:
[50]	valid_0's l2: 426.691
Training until

In [16]:
lgbm = LigthTrainer()
lgbm.fit(train,model_name="sin_declarado", to_drop=['researchable_id', 'estimate', 'declarativa','net_income_verified'], target="ingreso_neto_comprobado")
lgbm.X_train = train[lgbm.columns]
lgbm.y_train = train[lgbm.target].clip(0,100000)
lgbm.X_test = test_df[lgbm.columns]
lgbm.y_test = test_df[lgbm.target]

lgbm.optimize()
lgbm.evaluate()
lgbm.save_model("preds_sin_declarado")


Training until validation scores don't improve for 20 rounds
Early stopping, best iteration is:
[75]	valid_0's l2: 5.73038e+08
Training until validation scores don't improve for 20 rounds
Early stopping, best iteration is:
[81]	valid_0's l2: 5.74479e+08
Training until validation scores don't improve for 20 rounds
Early stopping, best iteration is:
[147]	valid_0's l2: 5.66346e+08
Training until validation scores don't improve for 20 rounds
Did not meet early stopping. Best iteration is:
[500]	valid_0's l2: 5.83369e+08
Training until validation scores don't improve for 20 rounds
Did not meet early stopping. Best iteration is:
[500]	valid_0's l2: 5.78115e+08
Training until validation scores don't improve for 20 rounds
Early stopping, best iteration is:
[392]	valid_0's l2: 5.6835e+08
Training until validation scores don't improve for 20 rounds
Did not meet early stopping. Best iteration is:
[500]	valid_0's l2: 5.76974e+08
Training until validation scores don't improve for 20 rounds
Early s